# Traveling Salesman problem

## Putting constrains on QUBO function

In this exercise you will learn basics of optimization performed on D-Wave quantum annealer. We start with the QUBO function:
$$F(x_i)=\sum_i^N a_ix_i + \sum_{i<j}^N b_{ij} x_i x_j$$


In order to understand how to map a problem into the QUBO function and how D-Wave finds its minimum, we first play with {$x_i$} vector of length three: {$x_1$, $x_2$, $x_3$}.
We put constrain on the sum of the $x_i$ elements to be $N$=0,1,2,3. One can realize this by the following QUBO function:
$$(\sum_{i=1}^{3}x_i-N)^2$$

It results in the following expansion: 

$$(\sum_{i=1}^{3}x_i-N)^2=(\sum_{i=1}^{3}x_i)^2-2N(\sum_{i=1}^{3}x_i)+N^2$$
$$=(\sum_{i=1}^{3}x_i^2)+2(\sum_{i>j}^{3}x_ix_j)-2N(\sum_{i=1}^{3}x_i)+N^2$$
$$=(1-2N)\sum_{i=1}^{3}x_i+2\sum_{i=1}^{3}x_ix_j$$


which gives the following $a_i$ and $b_{ij}$ coefficients:
$$a_i=1-2N$$
$$b_{ij}=2$$

so we deliver to D-wave the following $a$ vector and $b$ matrix

$$a=\begin{bmatrix} 
	1-2N \\
	1-2N \\
	1-2N \\
	\end{bmatrix}
	\quad$$

so, for example, for N=1
$$a=\begin{bmatrix} 
	-1 \\
	-1 \\
	-1 \\
	\end{bmatrix}
	\quad$$

$$b=\begin{bmatrix} 
	0 & 2 & 2 \\
	0 & 0 & 2\\
	0 & 0 & 0 \\
	\end{bmatrix}
	\quad$$

_Please run the code and see what happens._


In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite
import dwave.inspector

import numpy as np

def get_token():

    return 'INSERT YOUR TOKEN HERE'

Q = [[-1.,2.,2.],[0.,-1.,2.],[0.,0.,-1.]]

numruns = 100
sampler = EmbeddingComposite(DWaveSampler())
sample_set = sampler.sample_qubo(Q, annealing_time=3, num_reads=numruns)

print(sample_set)

dwave.inspector.show(sample_set)


Now, we would like to avoid a solution with $x_2$=$x_3$=1. 

_How to do this?_

We force larger parameter $b_{23}$. Please try 2.5, 3.0, 5.0, 10., 50. and see what will happen.

_Discuss the limitations..._


## Traveling Salesman problem


The essense of traveling salesman problem is to find a minimum cost (minimum distance) path for a salesman to visit all the cities.
The cost can be easily realized by having:
$$\sum_i a_ix_i=MIN,$$ 
which is the first term in the QUBO function. However, this is not enough, as it will simply result in $x_i=0$ as the minimum energy solution.

Correct mapping of the problem of traveling salesman to the QUBO function is not a straightforward thing. One of the strategy is to associate each connection between two cities to a specific $x_i$ parameter. Then for a minimum problem of 5 cities we will have the following mapping:

|---|A|B|C|D|E|
|---|---|---|---|---|---|
|A|---|$x_1$|$x_2$|$x_3$|$x_4$|
|B|---|---|$x_5$|$x_6$|$x_7$|
|C|---|---|---|$x_8$|$x_8$|
|D|---|---|---|---|$x_{10}$|

In such a construction, the constrain is to have for a sum of horizontal and vertical for a given city (e.g., B) equaling exactly 2, so the each city is only visited once and connected twice to other cities. Considering city A, this gives us the following constraint:

$x_1 + x_2 + x_3 + x_4 = 2$ which leads to $(x_1+x_2+x_3+x_4 -2)^2=0$

==> $\sum_{i=1,4}x_i + 2\sum_{i>j=1,4} x_i x_j - 4\sum_{i=1,4}x_i + 4 = MIN$

When one considers other cities, the following $a$ vector and $b$ matrix elements are obtained:
$$a_i=-6$$
and for all considered {$x_i,x_j$} combinations
$$b_{i,j}=2$$

The problem could be easily generalized to the case of $N$ cities, which you have provided in the python script below.

We must multiply this constrain by a well selected $\alpha$ parameter, e.g. 500, to put it on equall footing with the main $a_i$ elements reflecting the distances.
The best formulated problem for D-wave has non-zero values of $a$ vector and $b$ matrix having similar values, i.e. not varying by more that factor of ~5.


The following python script does it automatically for a given $N$.


The first complex problem we solve is the traveling between 5 German cities: Cologne, Berlin, Hamburg, Munich and Dresden. The distances between the cities are provided in the following distance chart:

|---|KLN|HAM|MUE|DRE|BER|
|---|---|---|---|---|---|
|KLN|---|432.|575.|570.|575.|
|HAM|---|---|790.|460.|290.| 
|MUE|---|---|---|570.|585.| 
|DRE|---|---|---|---|190.|

_Please use this example and compute the minimum energy path. Please mark the most favorable path on the appended map._


In [ ]:
def calcD(sample, bMat, aVec, Nt):

    dist = 0.0
    for i in range(Nt):
        dist += aVec[i] * sample[i]

    return dist

def getOc(sample, Nt):

    No = 0
    for i in range(Nt):
        No += sample[i]

    return No

def map(i0,j0,N):
      i=i0
      j=j0
      if j < i:
         j1=j
         j=i
         i=j1

      kk=(i-1)*N+j
      for k in range(1,i+1):
         kk=kk-k

      return kk

N=6
ALPHA=500.

aVec_filename = 'distances6.dat'
aVec = np.loadtxt(aVec_filename)

Nt = len(aVec)
Q={}
for i in range(1,Nt+1):
   Q[(i-1,i-1)] = aVec[i-1]

for i in range(1, N+1):
  for j in range(1,N+1):
     if j > i:
         i1=map(i,j,N)-1  
         Q[(i1,i1)]=Q.get((i1,i1),0)-6*ALPHA

     for j1 in range(1,N+1):         
         if i != j: 
             if i != j1:
                 if j != j1: 
                     Q[(map(i,j,N)-1,map(i,j1,N)-1)]=Q.get((map(i,j,N)-1,map(i,j1,N)-1),0)+1.*ALPHA

bMat=[[0]*Nt for i in range(Nt)]

for i in range(1,Nt+1):
     for j in range(i+1,Nt+1):
         Q[(i-1,j-1)]=Q.get((i-1,j-1),0)+Q.get((j-1,i-1),0)
         Q[(j-1,i-1)]=0.0
         bMat[i-1][j-1]=Q[(i-1,j-1)]     

numruns = 100
sampler = EmbeddingComposite(DWaveSampler())
sample_set = sampler.sample_qubo(Q, annealing_time=100, num_reads=numruns)

print(sample_set)

for sample in sample_set:
     if getOc(sample, Nt) == N:
         print(sample)
         print('D: ', calcD(sample, bMat, aVec, Nt))
         print('No: ', getOc(sample, Nt))
        

dwave.inspector.show(sample_set)


 <img src="Deutschland.png" width="600">

More complex example is the travel between 6, 10 and more cities.
The relevant examples are provided and are based on the following table of distances:

|---|KLN|HAM|MUE|DRE|BER|GOT|ERF|STU|MAG|FUL|
|---|---|---|---|---|---|---|---|---|---|---|
|KLN|---|432.|575.|570.|575.|290.|370.|375.|435.|270.|
|HAM|---|---|790.|460.|290.|260.|360.|660.|280.|405.|
|MUE|---|---|---|570.|585.|515.|415.|230.|525.|380.| 
|DRE|---|---|---|---|190.|360.|215.|515.|235.|365.|
|BER|---|---|---|---|---|330.|300.|635.|155.|450.|
|GOT|---|---|---|---|---|---|135.|400.|195.|145.|
|ERF|---|---|---|---|---|---|---|345.|175.|160.|
|STU|---|---|---|---|---|---|---|---|515.|260.|
|MAG|---|---|---|---|---|---|---|---|---|340.|


_Please run it and analyze the obtained solutions!_

Have fun with the D-wave optimization... 
